# TA-Lib Factors Analysis - 1month horizon

#### Good Indicators
###### Price oscillator +
###### Mean Rev 1M -
###### Volatility 3M -
###### ADX - 
###### DX - 
#### Borderline Indicators
###### ATR +
#### Bad Indicators
######

In [ ]:
#quantopian imports
from quantopian.research import run_pipeline
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import QTradableStocksUS

from quantopian.pipeline.factors import Latest
from quantopian.pipeline.data import morningstar, Fundamentals
from quantopian.pipeline.factors import CustomFactor, SimpleMovingAverage, AverageDollarVolume,SimpleBeta, Returns, RSI,EWMA
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.classifiers.fundamentals import Sector
from quantopian.pipeline.data.zacks import EarningsSurprises
from quantopian.pipeline.data import factset
from quantopian.pipeline.data.psychsignal import stocktwits

#Python imports
import math
import talib
import numpy as np
import pandas as pd
import pyfolio as pf
from scipy import stats
import matplotlib.pyplot as plt
from sklearn import linear_model, decomposition, ensemble, preprocessing, isotonic, metrics
from scipy.stats.mstats import winsorize
from zipline.utils.numpy_utils import (
    repeat_first_axis,
    repeat_last_axis,
)
from scipy.stats.mstats import gmean
from sklearn.cluster import SpectralClustering
 
from collections import Counter
import talib


In [ ]:
"""
HELPER FUNCTIONS
"""

def plus_dm_helper(high, low):
    """
    Returns positive directional movement. Abstracted for use with more complex factors

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI

    Parameters
    ----------
    high : np.array
        matrix of high prices
    low : np.array
        matrix of low prices

    Returns
    -------
    np.array : matrix of positive directional movement

    """
    # get daily differences between high prices
    high_diff = (high - np.roll(high, 1, axis=0))[1:]

    # get daily differences between low prices
    low_diff = (np.roll(low, 1, axis=0) - low)[1:]

    # matrix of positive directional movement
    return np.where(((high_diff > 0) | (low_diff > 0)) & (high_diff > low_diff), high_diff, 0.)

def minus_dm_helper(high, low):
    """
    Returns negative directional movement. Abstracted for use with more complex factors

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI

    Parameters
    ----------
    high : np.array
        matrix of high prices
    low : np.array
        matrix of low prices

    Returns
    -------
    np.array : matrix of negative directional movement

    """
    # get daily differences between high prices
    high_diff = (high - np.roll(high, 1, axis=0))[1:]

    # get daily differences between low prices
    low_diff = (np.roll(low, 1, axis=0) - low)[1:]

    # matrix of megative directional movement
    return np.where(((high_diff > 0) | (low_diff > 0)) & (high_diff < low_diff), low_diff, 0.)


def trange_helper(high, low, close):
    """
    Returns true range

    http://www.macroption.com/true-range/

    Parameters
    ----------
    high : np.array
        matrix of high prices
    low : np.array
        matrix of low prices
    close: np.array
        matrix of close prices

    Returns
    -------
    np.array : matrix of true range

    """
    # define matrices to be compared
    close = close[:-1]
    high = high[1:]
    low = low[1:]

    # matrices for comparison
    high_less_close = high - close
    close_less_low = close - low
    high_less_low = high - low

    # return maximum value for each cel
    return np.maximum(high_less_close, close_less_low, high_less_low)

## RSI, CCI, MFI, MACD

In [ ]:
class CCI(CustomFactor):
    """
    Commodity Channel Index

    Momentum indicator

    **Default Inputs:** USEquityPricing.close, USEquityPricing.high, USEquityPricing.low

    **Default Window Length:** 14

    http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:commodity_channel_index_cci
    """

    inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
    window_length = 21

    def compute(self, today, assets, out, high, low, close):

        # typical price matrix
        typical_prices = (high + low + close) / 3.

        # mean of each column
        mean_typical = np.nanmean(typical_prices, axis=0)

        # mean deviation
        mean_deviation = np.sum(np.abs(typical_prices - np.tile(mean_typical, (len(typical_prices), 1))), axis=0) / self.window_length

        # CCI
        out[:] = (typical_prices[-1] - mean_typical) / (.015 * mean_deviation)
        

        
class MFI(CustomFactor):
    """
    Money Flow Index

    Volume Indicator

    **Default Inputs:**  USEquityPricing.high, USEquityPricing.low, USEquityPricing.close, USEquityPricing.volume

    **Default Window Length:** 15 (14 + 1-day for difference in prices)

    http://www.fmlabs.com/reference/default.htm?url=MoneyFlowIndex.htm
    """	    

    inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close, USEquityPricing.volume]
    window_length = 22

    def compute(self, today, assets, out, high, low, close, vol):

        # calculate typical price
        typical_price = (high + low + close) / 3.

        # calculate money flow of typical price
        money_flow = typical_price * vol

        # get differences in daily typical prices
        tprice_diff = (typical_price - np.roll(typical_price, 1, axis=0))[1:]

        # create masked arrays for positive and negative money flow
        pos_money_flow = np.ma.masked_array(money_flow[1:], tprice_diff < 0, fill_value = 0.)
        neg_money_flow = np.ma.masked_array(money_flow[1:], tprice_diff > 0, fill_value = 0.)

        # calculate money ratio
        money_ratio = np.sum(pos_money_flow, axis=0) / np.sum(neg_money_flow, axis=0)

        # MFI
        out[:] = 100. - (100. / (1. + money_ratio))
        
class ADX(CustomFactor):
    """
    Average Directional Movement Index

    Momentum indicator. Smoothed DX

    **Default Inputs:** USEquityPricing.high, USEquityPricing.low, USEquitypricing.close

    **Default Window Length:** 29

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI
    """	    
    inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
    window_length = 29

    def compute(self, today, assets, out, high, low, close):

        # positive directional index
        plus_di = 100 * np.cumsum(plus_dm_helper(high, low) / trange_helper(high, low, close), axis=0)

        # negative directional index
        minus_di = 100 * np.cumsum(minus_dm_helper(high, low) / trange_helper(high, low, close), axis=0)

        # full dx with 15 day burn-in period
        dx_frame = (np.abs(plus_di - minus_di) / (plus_di + minus_di) * 100.)[14:]

        # 14-day EMA
        span = 14.
        decay_rate = 2. / (span + 1.)
        weights = weights_long = np.full(span, decay_rate, float) ** np.arange(span + 1, 1, -1)

        # return EMA
        out[:] = np.average(dx_frame, axis=0, weights=weights)


def APO(shortperiod=12, longperiod=26):
    """
    Absolute Price Oscillator

    Momentum indeicator. Difference between EWMAs (exponential weighted moving averages) of 
    short and long periods.

    **Default Inputs:** 12, 26

    **Default Window Length:** 46 (26 + 20-day burn-in)

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/apo

    Parameters
    ----------
    shortperiod : int > 0
        window length for short EWMA
    longperiod : int > 0 (>= shortperiod)
        window length for longer EWMA

    Returns
    -------
    zipline.Factor
    """
    buffer_window = longperiod + 20

    fast = EWMA.from_span(inputs=[USEquityPricing.close], window_length=buffer_window, span=shortperiod)
    slow = EWMA.from_span(inputs=[USEquityPricing.close], window_length=buffer_window, span=longperiod) 
    return fast - slow


class ATR(CustomFactor):
    """
    Average True Range

    Momentum indicator

    **Default Inputs:** USEquityPricing.high, USEquityPricing.low, USEquityPricing.close

    **Default Window Length:** 15 (14+1)

    https://en.wikipedia.org/wiki/Average_true_range
    """
    inputs=[USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
    window_length = 21

    def compute(self, today, assets, out, high, low, close):

        tr_frame = trange_helper(high, low, close)
        decay_rate= 2./(len(tr_frame) + 1.)
        weights = np.full(len(tr_frame), decay_rate, float) ** np.arange(len(tr_frame) + 1, 1, -1)
        out[:] = np.average(tr_frame, axis=0, weights=weights)


In [ ]:
def make_pipeline():
    ZSCORE_FILTER = 3 # Maximum number of standard deviations to include before counting as outliers
    ZERO_FILTER = 0.001 # Minimum weight we allow before dropping security
    
    # ALPHA FACTOR 1
    alpha_factor1 = CCI()
    
    # Standardized logic for each input factor after this point
    alpha_w1 = alpha_factor1.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor1.isfinite())
    
    alpha_z1 = alpha_w1.zscore()
    alpha_weight1 = alpha_z1 / 100.0
    
    outlier_filter1 = alpha_z1.abs() < ZSCORE_FILTER
    zero_filter1 = alpha_weight1.abs() > ZERO_FILTER
 
    universe1 = QTradableStocksUS() & \
               outlier_filter1 & \
               zero_filter1
    
    # ALPHA FACTOR 2
    alpha_factor2 = MFI()
    
    # Standardized logic for each input factor after this point
    alpha_w2 = alpha_factor2.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor2.isfinite())
    
    alpha_z2 = alpha_w2.zscore()
    alpha_weight2 = alpha_z2 / 100.0
    
    outlier_filter2 = alpha_z2.abs() < ZSCORE_FILTER
    zero_filter2 = alpha_weight2.abs() > ZERO_FILTER
 
    universe2 = QTradableStocksUS() & \
               outlier_filter2 & \
               zero_filter2
    
    # ALPHA FACTOR 3
    alpha_factor3 = ADX()
    
    # Standardized logic for each input factor after this point
    alpha_w3 = alpha_factor3.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor3.isfinite())
    
    alpha_z3 = alpha_w3.zscore()
    alpha_weight3 = alpha_z3 / 100.0
    
    outlier_filter3 = alpha_z3.abs() < ZSCORE_FILTER
    zero_filter3 = alpha_weight3.abs() > ZERO_FILTER
 
    universe3 = QTradableStocksUS() & \
               outlier_filter3 & \
               zero_filter3
    
    # ALPHA FACTOR 4
    alpha_factor4 = APO()
    
    # Standardized logic for each input factor after this point
    alpha_w4 = alpha_factor4.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor4.isfinite())
    
    alpha_z4 = alpha_w4.zscore()
    alpha_weight4 = alpha_z4 / 100.0
    
    outlier_filter4 = alpha_z4.abs() < ZSCORE_FILTER
    zero_filter4 = alpha_weight4.abs() > ZERO_FILTER
 
    universe4 = QTradableStocksUS() & \
               outlier_filter4 & \
               zero_filter4
    
    # ALPHA FACTOR 5
    alpha_factor5 = ATR()
    
    # Standardized logic for each input factor after this point
    alpha_w5 = alpha_factor5.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor5.isfinite())
    
    alpha_z5 = alpha_w5.zscore()
    alpha_weight5 = alpha_z5 / 100.0
    
    outlier_filter5 = alpha_z5.abs() < ZSCORE_FILTER
    zero_filter5 = alpha_weight5.abs() > ZERO_FILTER
 
    universe5 = QTradableStocksUS() & \
               outlier_filter5 & \
               zero_filter5
    
    universe =  universe1 & universe2 & universe3 & universe4 & universe5
    
    alpha_weight = alpha_weight2 + alpha_weight3 + alpha_weight4 +alpha_weight5
    
    testing_quantiles = alpha_weight.quantiles(2)
    
    sector = Sector()
    
    pipe = Pipeline(
        columns={
            'alpha_weight': alpha_weight,
            'shorts':testing_quantiles.eq(0),
            'longs':testing_quantiles.eq(1),
            'CCI': alpha_weight1,
            'MFI': alpha_weight2,
            'ADX': alpha_weight3,
            'APO': alpha_weight4,
            'ATR': alpha_weight5,
            'sector': sector,
        },
        screen=universe
    )
    return pipe

In [ ]:
result = run_pipeline(make_pipeline(), start_date = '2015-01-01', end_date = '2016-01-01')
result.head()

In [ ]:
assets = result.index.levels[1]
len(assets)

In [ ]:
pricing_data = get_pricing(assets, start_date = '2014-01-01', end_date = '2017-01-01',fields='open_price')

#### CCI - Weak

In [ ]:
import alphalens
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

sector_labels, sector_labels[-1] = dict(Sector.SECTOR_NAMES), "Unknown"

factor_data1 = get_clean_factor_and_forward_returns(
    result['CCI'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### MFI - Weak

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['MFI'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### ADX - good negative

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['ADX'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### APO - weak

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['APO'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### ATR - borderline

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['ATR'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

## Price Oscillator, MeanRev1M, Volatility3M

In [ ]:
class PriceOscillator(CustomFactor):
    inputs = (USEquityPricing.close,)
    window_length = 252

    def compute(self, today, assets, out, close):
        four_week_period = close[-20:]
        np.divide(
            np.nanmean(four_week_period, axis=0),
            np.nanmean(close, axis=0),
            out=out,
        )
        out -= 1
        
class MeanReversion1M(CustomFactor):
    inputs = (Returns(window_length=21),)
    window_length = 252

    def compute(self, today, assets, out, monthly_rets):
        np.divide(
            monthly_rets[-1] - np.nanmean(monthly_rets, axis=0),
            np.nanstd(monthly_rets, axis=0),
            out=out,
        )

class Volatility3M(CustomFactor):
    inputs = [Returns(window_length=2)]
    window_length = 63

    def compute(self, today, assets, out, rets):
        np.nanstd(rets, axis=0, out=out)
        

In [ ]:
def make_pipeline():
    ZSCORE_FILTER = 3 # Maximum number of standard deviations to include before counting as outliers
    ZERO_FILTER = 0.001 # Minimum weight we allow before dropping security
               
    # ALPHA FACTOR 2
    alpha_factor2 = PriceOscillator()
    
    # Standardized logic for each input factor after this point
    alpha_w2 = alpha_factor2.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor2.isfinite())
    
    alpha_z2 = alpha_w2.zscore()
    alpha_weight2 = alpha_z2 / 100.0
    
    outlier_filter2 = alpha_z2.abs() < ZSCORE_FILTER
    zero_filter2 = alpha_weight2.abs() > ZERO_FILTER
 
    universe2 = QTradableStocksUS() & \
               outlier_filter2 & \
               zero_filter2
    
    # ALPHA FACTOR 3
    alpha_factor3 = MeanReversion1M()
    
    # Standardized logic for each input factor after this point
    alpha_w3 = alpha_factor3.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor3.isfinite())
    
    alpha_z3 = alpha_w3.zscore()
    alpha_weight3 = alpha_z3 / 100.0
    
    outlier_filter3 = alpha_z3.abs() < ZSCORE_FILTER
    zero_filter3 = alpha_weight3.abs() > ZERO_FILTER
 
    universe3 = QTradableStocksUS() & \
               outlier_filter3 & \
               zero_filter3
    
    # ALPHA FACTOR 4
    alpha_factor4 = Volatility3M()
    
    # Standardized logic for each input factor after this point
    alpha_w4 = alpha_factor4.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor4.isfinite())
    
    alpha_z4 = alpha_w4.zscore()
    alpha_weight4 = alpha_z4 / 100.0
    
    outlier_filter4 = alpha_z4.abs() < ZSCORE_FILTER
    zero_filter4 = alpha_weight4.abs() > ZERO_FILTER
 
    universe4 = QTradableStocksUS() & \
               outlier_filter4 & \
               zero_filter4
    
    
    universe =  universe2 & universe3 & universe4
    
    alpha_weight = alpha_weight2 + alpha_weight3 + alpha_weight4
    
    testing_quantiles = alpha_weight.quantiles(2)
    
    sector = Sector()
    
    pipe = Pipeline(
        columns={
            'alpha_weight': alpha_weight,
            'shorts':testing_quantiles.eq(0),
            'longs':testing_quantiles.eq(1),
            'price_oscillator': alpha_weight2,
            'mean_rev': alpha_weight3,
            'vol_3m': alpha_weight4,
            'sector': sector,
        },
        screen=universe
    )
    return pipe

In [ ]:
result = run_pipeline(make_pipeline(), start_date = '2015-01-01', end_date = '2016-01-01')
result.head()

In [ ]:
assets = result.index.levels[1]
len(assets)

In [ ]:
pricing_data = get_pricing(assets, start_date = '2012-01-01', end_date = '2018-01-01', fields='open_price')

#### price_oscillator - GOOD

In [ ]:
import alphalens
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

sector_labels, sector_labels[-1] = dict(Sector.SECTOR_NAMES), "Unknown"

factor_data1 = get_clean_factor_and_forward_returns(
    result['price_oscillator'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### mean_rev - good negative

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['mean_rev'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### vol_3m - Good negative

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['vol_3m'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

## AD, WILLR, CMO,DX

In [ ]:
class AD(CustomFactor):
    """
    Chaikin Accumulation Distribution Line

    Volume indicator

    **Default Inputs:** USEquityPricing.high, USEquityPricing.low, USEquitypricing.close, USEquityPricing.volume

    **Default Window Length:** 14

    http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:accumulation_distribution_line
    """	    
    inputs = [USEquityPricing.close, USEquityPricing.high, USEquityPricing.low, USEquityPricing.volume]
    window_length = 14

    def compute(self, today, assets, out, close, high, low, vol):

        # close location value
        clv = ((close - low) - (high - close)) / (high - low)
        ad = clv * vol
        out[:] =  np.sum(ad, axis=0) 
        
class WILLR(CustomFactor):
    """
    Williams %R

    **Default Inputs:**  USEquityPricing.high, USEquityPricing.low, USEquityPricing.close

    **Default Window Length:** 14

    http://www.fmlabs.com/reference/default.htm?url=WilliamsR.htm
    """    	    
    inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
    window_length = 21

    def compute(self, today, assets, out, high, low, close):
        out[:] = (np.nanmax(high, axis=0) - close[-1]) / (np.nanmax(high, axis=0) - np.nanmin(low, axis=0)) * -100.


class CMO(CustomFactor):
    """
    Chande Momentum Oscillator

    Momentum indicator. Descriptor of overought/oversold conditions

    **Default Inputs:** USEquityPricing.close

    **Default Window Length:** 15 (14 + 1-day for differences)

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/cmo
    """
    inputs=[USEquityPricing.close]
    window_length = 22

    def compute(self, today, assets, out, close):

        # daily differences in close prices
        close_diff = (close - np.roll(close, 1, axis=0))[1:]

        # close differences on up days
        su = np.sum(np.where(close_diff > 0, close_diff, 0), axis=0)

        # absolute value of close differences on down days
        sd = np.abs(np.sum(np.where(close_diff < 0, close_diff, 0), axis=0))

        # CMO
        out[:] = 100 * (su - sd) / (su + sd)

class DX(CustomFactor):
    """
    Directional Movement Index

    Momentum indicator

    **Default Inputs:** USEquityPricing.high, USEquityPricing.low, USEquitypricing.close

    **Default Window Length:** 15

    https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/DMI
    """	    
    inputs = [USEquityPricing.high, USEquityPricing.low, USEquityPricing.close]
    window_length = 22

    def compute(self, today, assets, out, high, low, close):

        # positive directional index
        plus_di = 100 * np.sum(plus_dm_helper(high, low) / (trange_helper(high, low, close)), axis=0)

        # negative directional index
        minus_di = 100 * np.sum(minus_dm_helper(high, low) / (trange_helper(high, low, close)), axis=0)

        # DX
        out[:] = np.abs(plus_di - minus_di) / (plus_di + minus_di) * 100.


In [ ]:
def make_pipeline():
    ZSCORE_FILTER = 3 # Maximum number of standard deviations to include before counting as outliers
    ZERO_FILTER = 0.001 # Minimum weight we allow before dropping security
    
    # ALPHA FACTOR 1
    alpha_factor1 = AD()
 
    # Standardized logic for each input factor after this point
    alpha_w1 = alpha_factor1.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor1.isfinite())
 
    alpha_z1 = alpha_w1.zscore()
    alpha_weight1 = alpha_z1 / 100.0
 
    outlier_filter1 = alpha_z1.abs() < 3
    zero_filter1 = alpha_weight1.abs() > 0.001
    
    universe1 = QTradableStocksUS() & \
               outlier_filter1 & \
               zero_filter1
               
    # ALPHA FACTOR 2
    alpha_factor2 = WILLR()
    
    # Standardized logic for each input factor after this point
    alpha_w2 = alpha_factor2.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=QTradableStocksUS() & alpha_factor2.isfinite())
    
    alpha_z2 = alpha_w2.zscore()
    alpha_weight2 = alpha_z2 / 100.0
    
    outlier_filter2 = alpha_z2.abs() < ZSCORE_FILTER
    zero_filter2 = alpha_weight2.abs() > ZERO_FILTER
 
    universe2 = QTradableStocksUS() & \
               outlier_filter2 & \
               zero_filter2
    
    # ALPHA FACTOR 3
    alpha_factor3 = CMO()
    
    # Standardized logic for each input factor after this point
    alpha_w3 = alpha_factor3.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor3.isfinite())
    
    alpha_z3 = alpha_w3.zscore()
    alpha_weight3 = alpha_z3 / 100.0
    
    outlier_filter3 = alpha_z3.abs() < ZSCORE_FILTER
    zero_filter3 = alpha_weight3.abs() > ZERO_FILTER
 
    universe3 = QTradableStocksUS() & \
               outlier_filter3 & \
               zero_filter3
    
    # ALPHA FACTOR 4
    alpha_factor4 = DX()
    
    # Standardized logic for each input factor after this point
    alpha_w4 = alpha_factor4.winsorize(min_percentile=0.02,
                                     max_percentile=0.98,
                                     mask=alpha_factor4.isfinite())
    
    alpha_z4 = alpha_w4.zscore()
    alpha_weight4 = alpha_z4 / 100.0
    
    outlier_filter4 = alpha_z4.abs() < ZSCORE_FILTER
    zero_filter4 = alpha_weight4.abs() > ZERO_FILTER
 
    universe4 = QTradableStocksUS() & \
               outlier_filter4 & \
               zero_filter4
    
    
    universe = universe1 & universe2 & universe3 & universe4
    
    alpha_weight = alpha_weight1 + alpha_weight2 + alpha_weight3 + alpha_weight4
    
    testing_quantiles = alpha_weight.quantiles(2)
    
    sector = Sector()
    
    pipe = Pipeline(
        columns={
            'alpha_weight': alpha_weight,
            'shorts':testing_quantiles.eq(0),
            'longs':testing_quantiles.eq(1),
            'AD': alpha_weight1,
            'WILLR': alpha_weight2,
            'CMO': alpha_weight3,
            'DX': alpha_weight4,
            'sector': sector,
        },
        screen=universe
    )
    return pipe

In [ ]:
result = run_pipeline(make_pipeline(), start_date = '2015-01-01', end_date = '2016-01-01')
result.head()

In [ ]:
assets = result.index.levels[1]
len(assets)

In [ ]:
pricing_data = get_pricing(assets, start_date = '2014-01-01', end_date = '2017-01-01', fields='open_price')

#### AD - Weak

In [ ]:
import alphalens
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet

sector_labels, sector_labels[-1] = dict(Sector.SECTOR_NAMES), "Unknown"

factor_data1 = get_clean_factor_and_forward_returns(
    result['AD'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### WILLR - weak

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['WILLR'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### CMO - weak

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['CMO'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)

#### DX - Good negative

In [ ]:
factor_data1 = get_clean_factor_and_forward_returns(
    result['DX'],
    pricing_data,
    quantiles =2,
    periods = (21,63,126),
    groupby=result['sector'],
    groupby_labels=sector_labels,
)

create_full_tear_sheet(factor_data1, by_group=True)